# Lab Test
This notebook is dedicated to clean the data from lab tests into the [record format](http://160.89.29.120:30046/edit/src/main.py) as we defined for each data source. It includes the following sections:
1. Read the files and filter the tests
2. Cleaning
3. Labeling
4. Combine and write to disk

## 1. Read the file and filter the tests

Import the libraries

In [ ]:
import pandas as pd 
import os
import numpy as np
import util.cleaning_tools as tools
%load_ext autoreload
%autoreload 2

Read the file

In [ ]:
# define the file path and tables path for file reading
file_path = r'../DATAFILE'
tid_to_eid_path = r'iams_entity_concept'
labresult_cps_path = 'lis_cps_result_data'
labresult_hms_path = 'lis_hms_result_data'

# read the fragment files and concat them
labresult_cps = tools.fileReader(file_path, labresult_cps_path)
labresult_hms = tools.fileReader(file_path, labresult_hms_path)
tid_to_eid = tools.fileReader(file_path, tid_to_eid_path)

# concat them
labresult = pd.concat([labresult_cps, labresult_hms], axis=0)
# delete the reference to the raw data for the sake of garbage collection
del labresult_cps
del labresult_hms

Since the combined lab test table large is quiet large, I will first filter the target tests. 

In [ ]:
#define the target term_id and find out the corresponding mapping table
target_tid = [5200485,5203348,5200788,5201993,5204343,5203255,5200306]
target_id = pd.Series(target_tid).rename('term_id')
target_tid_mapping = pd.merge(target_id,tid_to_eid,how='left',on='term_id')
target_tid_mapping

In [ ]:
# join the mapping table and labresult on the entity_id
labresult_filtered = pd.merge(labresult, target_tid_mapping, how='inner',on='entity_id')

This cell checks on the lab test unit applied on different target tests.

In [ ]:
labresult_filtered[["term_id", "si_unit", "test_unit"]].drop_duplicates().reset_index(drop=True)

In [ ]:
#drop duplicates just in case there is any duplicates records
labresult_filtered = labresult_filtered.drop_duplicates()
labresult_filtered

In [ ]:
# write to csv file
labresult_filtered.to_csv(r'../tables/output/labresult_filtered.csv')
# labresult_filtered = pd.read_csv(r'../tables/output/labresult_filtered.csv')

In [ ]:
#check the test unit for each kind of term_id
labresult_filtered.groupby('term_id',as_index=False)["term_id","si_unit"].apply(lambda x : x.head(1)).reset_index(drop=True)

## Cleaning

The cell is deprecated becasue I will use standard numeric result to be consistent with the following analysis.

In [ ]:
# # extract the numeric part of the test result 
# pattern = r'([0-9/.]*)'
# labresult_filtered['numeric_test_result'] = labresult_filtered['result_str'].str.extract(pattern)
# # transform the test result to numeric
# labresult_filtered.numeric_test_result = pd.to_numeric(labresult_filtered.numeric_test_result,downcast='float')
# labresult_filtered[['numeric_test_result','result_str']]

In [ ]:
# the raw data record the null value as "" replace it with numpy null value class
labresult_filtered["si_numeric"].replace('""', np.nan, inplace=True)
labresult_filtered["si_numeric"].isnull().sum()

I drop the null records since there is no reasonable explanation of the missing values.

In [ ]:
labresult_filtered = labresult_filtered.query("si_numeric.notnull()", engine='python')
# after the filtering, we can safely cast the type to float with numpy built-in class np.float
labresult_filtered["si_numeric"] = labresult_filtered["si_numeric"].astype(np.float)
labresult_filtered.reset_index(drop=True, inplace=True)

## Labeling

Here we determine the type of result of each record and assign the result to a new column diab_type.
There are three possible results:
1. Diabetes(diab)
2. Pre-Diabete(pre)
3. Normal(normal)

In [ ]:
#define the pre-diabetes test result range
r_range = {5200485: [5.7, 6.4],
           5203348: [39, 46],
           5200788: [7.8, 11.1],
           5201993: [7.8, 11.1],
           5204343: [5.6, 6.9],
           5203255: [5.6, 6.9],
           5200306: [5.6, 6.9]
          }

diab_type = []
# loop over each record, it expects no built-in performance optimization
for i in range(labresult_filtered.shape[0]):
    term_id = labresult_filtered.iloc[i].loc["term_id"]
    rg = r_range[term_id]
    test_result = labresult_filtered.loc[i, "si_numeric"]
    if test_result > rg[1]:
        # diabetes test result
        diab_type.append("diab")
    elif test_result <= rg[1] and test_result >= rg[0]:
        # pre-diabetes result
        diab_type.append("pre")
    else:
        # 
        diab_type.append("normal")

labresult_filtered = labresult_filtered.assign(diab_type = diab_type)

In [ ]:
tools.getNum(labresult_filtered[labresult_filtered.diab_type == "pre"],r=False)
tools.getNum(labresult_filtered[labresult_filtered.diab_type == "diab"],r=False)

In [ ]:
diab_records = labresult_filtered[labresult_filtered.diab_type == "diab"]

 ### Diabetes confirmation

This section determines the diabetes record based on the criterial provided: only include the patients who has two abnormal test of diabetes within 12 weeks for diabete patients or has a least one abnormal HbA1c test 

In [ ]:
# Only include the patients who has two abnormal test of diabetes within 12 weeks 
# for diabetes patients or has a least one abnormal HbA1C test 


# def diagConfirmed(df,h=7*24*12):
#     # sort by reference_dtm
#     df = df.sort_values("diff_in_hour_reference_dtm")
#     l = df.shape[0]
#     i = 0
#     if l == 1 and df.iloc[0, 3] in [5200485, 5203348]:
#         return df.iloc[0,:]
#     while(i < l-1):
#         if (df.iloc[i+1,2] - df.iloc[i,2]) <= h or df.iloc[i, 3] in [5200485, 5203348] :
#             # return the earliest record of the diabetes
#             return df.iloc[i,:]
#         i += 1
#     return None


def diagConfirmed(df:pd.DataFrame, h:int = 7*24*12) -> pd.DataFrame:
    '''
    callable instance that will be applied on each grouped dataframe, 
    this method will implement the diabetes matching rules: include the 
    patients who has two abnormal test of diabetes within 12 weeks 
    for diabetes patients or has a least one abnormal HbA1C test. 
    
    Args:
        df: pandas dataframe that is already grouped by some keys.
        h: maximum time interval in hours for two abnormal tests except for HBA1C, default 12 weeks.
    Return:
        the earliest record of confirmed diabetes of this patients.
    '''
    l = df.shape[0]
    for i in range(l):
        if df.iloc[i]["term_id"] in [5200485, 5203348]: #HbA1c tests
            return df.iloc[i,:]
        elif i < l-1:
            if (df.iloc[i+1].loc["diff_in_hour_reference_dtm"] - df.iloc[i].loc["diff_in_hour_reference_dtm"]) <= h:
                return df.iloc[i,:]
        else:
            return None

diab_patients = diab_records\
                .sort_values(["pseudo_patient_key", "diff_in_hour_reference_dtm"])\
                .groupby(by=["pseudo_patient_key"])\
                .apply(diagConfirmed)
diab_patients.reset_index(drop=True, inplace=True)
diab_patients.dropna(inplace=True)

### Pre-diabetes confirmation

In [ ]:
# find the earliest confirmination of pre-diabetes
l = ["pseudo_patient_key","reference_dtm", "diff_in_hour_reference_dtm", "term_id","diab_type"]
pre_records = labresult_filtered[labresult_filtered.diab_type == "pre"][l]
pre_patients = pre_records\
            .sort_values(["pseudo_patient_key","diff_in_hour_reference_dtm"])\
            .groupby(by=["pseudo_patient_key"])\
            .apply(lambda x : x.head(1))

## Combine and write to disk
Combine the prediabetes and diabetes records and write to disk.

In [ ]:
combine_records = pd.concat([pre_patients, diab_patients])
# reset the index
combine_records.reset_index(drop=True, inplace=True) 
combine_records.dropna(inplace=True) # drop null value

In [ ]:
# rename to fit the common fields naming rules
combine_records.rename({"reference_dtm":"dx_dtm", "diff_in_hour_reference_dtm": "diff_hour"}, axis="columns", inplace=True)

In [ ]:
# add src
combine_records["src"] = "lab"
# write to csv file
combine_records[["pseudo_patient_key", "dx_dtm", "diff_hour", "diab_type", "src"]].to_csv(r"../tables/output/first_diag_lab.csv")

In [ ]:
combine_records.groupby("diab_type").count()